# ContentBased filtering in E-commerce

## Personal Information
- Fullname: Dương Nhật Minh
- Email: nhatminh.cdt@gmail.com

In [ ]:
# from google.colab import drive
# drive.mount("/content/gdrive", force_remount=True)

# %cd '/content/gdrive/My Drive/DS_ML/LDS0/LDS0_k283_ONLINE_DuongNhatMinh/Project2/'

In [ ]:
# !pip install underthesea
# !pip install multiprocess
# !pip install demoji
# !pip install pyvi
# !pip install dask

## Reference
- [Underthesea](https://github.com/undertheseanlp/underthesea)

## Import libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# For multiprocessing
import os
import ast
import time

In [3]:
# display rows of data with whole text
pd.options.display.max_colwidth = None

In [4]:
DataPath          = 'Data/'
# FileName          = 'Products_ThoiTrangNam_raw.csv'
ProcessedFileName = 'Products_ThoiTrangNam_rating_processed.csv'
UserRecFileName   = 'UsrRecMatrix_.csv'
ItemRecFileName   = 'ItemRecMatrix_.csv'
# join path and file name
# FilePath          = os.path.join(DataPath, FileName)
ProcessedFilePath = os.path.join(DataPath, ProcessedFileName)
UserRecFilePath   = os.path.join(DataPath, UserRecFileName)
ItemRecFilePath   = os.path.join(DataPath, ItemRecFileName)

## Load Data

In [5]:
df = pd.read_csv(ProcessedFilePath, encoding='utf8', header=0, sep='\t')
df.head()

,product_id,user_id,user,rating
0,190,1,karmakyun2nd,5
1,190,2,tranquangvinh_vv,5
2,112544,2,tranquangvinh_vv,5
3,190,3,nguyenquoctoan2005,5
4,190,4,nguyenthuyhavi,5


In [7]:
df_user = pd.read_csv(UserRecFilePath, encoding='utf8', header=0)
df_user.head(3)

,user_id,product_id,rating
0,1,172103,6.515820
1,1,211592,6.399804
2,1,173725,6.389039


In [16]:
df_item = pd.read_csv(ItemRecFilePath, encoding='utf8', header=0)
df_item.head(3)

,product_id,user_id,rating
0,101,11590,6.319169
1,101,14215,6.219183
2,101,11964,6.212409


In [14]:
# Get unique user_id and sort by user_id
userIds = df_user['user_id'].unique()
userIds.sort()
userIds

array([     1,      2,      3, ..., 650632, 650635, 650636], dtype=int64)

In [15]:
print(df_user['user_id'].unique().shape)

(479997,)


In [18]:
print(df_item['product_id'].unique().shape)

(30001,)


In [7]:
# Get information of product with product_id from df_user
def get_product_info(product_id):

,"user_id,recommendations"
0,"(31, [Row(product_id=21799, rating=6.279116153717041), Row(product_id=141451, rating=6.21476411819458), Row(product_id=15798, rating=6.191742897033691), Row(product_id=12964, rating=6.123527526855469), Row(product_id=172495, rating=6.121673583984375), Row(product_id=174838, rating=6.105851173400879), Row(product_id=19309, rating=6.101972579956055), Row(product_id=19918, rating=6.09112024307251), Row(product_id=24208, rating=6.084497928619385), Row(product_id=171390, rating=6.068795204162598)])"
1,"(34, [Row(product_id=15674, rating=7.0020294189453125), Row(product_id=174838, rating=6.912172317504883), Row(product_id=232790, rating=6.858043193817139), Row(product_id=233263, rating=6.835661888122559), Row(product_id=19918, rating=6.829843521118164), Row(product_id=121218, rating=6.782854080200195), Row(product_id=15880, rating=6.764533996582031), Row(product_id=151326, rating=6.7391357421875), Row(product_id=12548, rating=6.71539306640625), Row(product_id=12382, rating=6.693021774291992)])"
2,"(53, [Row(product_id=11649, rating=5.839995384216309), Row(product_id=151237, rating=5.777667999267578), Row(product_id=11842, rating=5.7586870193481445), Row(product_id=233707, rating=5.681207180023193), Row(product_id=123945, rating=5.669703483581543), Row(product_id=232120, rating=5.594894886016846), Row(product_id=123408, rating=5.580158710479736), Row(product_id=101464, rating=5.544166088104248), Row(product_id=26118, rating=5.543569564819336), Row(product_id=232233, rating=5.534534454345703)])"


In [9]:
df_ = []
# iterate over each row and extract the relevant values
for i, row in df_user.iterrows():
  for j, value in enumerate(row['user_id,recommendations']):
    print(f'i={i}, row={row}, j={j}, value={value}')
    # df_.append({'user_id': row['user_id'], 'product_id': value['product_id'], 'rating': value['rating']})
    break
  break

i=0, row=user_id,recommendations    (31, [Row(product_id=21799, rating=6.279116153717041), Row(product_id=141451, rating=6.21476411819458), Row(product_id=15798, rating=6.191742897033691), Row(product_id=12964, rating=6.123527526855469), Row(product_id=172495, rating=6.121673583984375), Row(product_id=174838, rating=6.105851173400879), Row(product_id=19309, rating=6.101972579956055), Row(product_id=19918, rating=6.09112024307251), Row(product_id=24208, rating=6.084497928619385), Row(product_id=171390, rating=6.068795204162598)])
Name: 0, dtype: object, j=0, value=31


In [ ]:
df_item = pd.read_csv(ItemRecFilePath, encoding='utf8', header=0, sep='\t')
df_item.head(3)

## Data Overview

In [ ]:
df_user.columns

In [ ]:
# explode the list of recommended items into rows
df_ = df_user.explode('user_id')
df_.head(3)

In [ ]:
df.tail(3)

## Data Preprocessing

In [ ]:
### Combine product name and product description
df['product_name_description'] = df['product_name'] + ' ' + df['description']
df.product_name_description.head(3)

### Drop null values

In [ ]:
print('Data shape:', df.shape)
missing_val_num = df.isnull().sum()
missing_percent = (missing_val_num / df.shape[0]) * 100
missing_info = {'missing_values': missing_val_num,
                'missing_percent': round(missing_percent,3)}
missing_df = pd.DataFrame(missing_info)
missing_df

In [ ]:
# Drop rows if 'product_id', 'sub_category', or 'product_name_description' has NaN values
df.dropna(subset=['product_id', 'sub_category', 'product_name_description'], inplace=True)
print('Data shape:', df.shape)
missing_val_num = df.isnull().sum()
missing_percent = (missing_val_num / df.shape[0]) * 100
missing_info = {'missing_values': missing_val_num,
                'missing_percent': round(missing_percent,3)}
missing_df = pd.DataFrame(missing_info)
missing_df

### Check duplicate data

In [ ]:
# Display duplicate rows
print('Duplicate data:', df.duplicated().sum())
# df[df.duplicated()]
data_temp = df[df.duplicated(keep=False)].sort_values(by='product_id')
data_temp.head()

=> No duplicate rows

### Add ID for sub-category

In [ ]:
# Display number of sub-category
print('Number of sub-category:', df['sub_category'].nunique())
# Check and display distribution of sub-category
plt.figure(figsize=(10,5))
sns.countplot(y='sub_category', data=df)
# Display number of products in each sub-category on the bar chart
for p in plt.gca().patches:
    plt.gca().text(p.get_width()+0.2, p.get_y()+0.55*p.get_height(),
                   '{:1.0f}'.format(p.get_width()), fontsize=11, color='dimgrey')

plt.gca().set_xlabel('Number of products', fontsize=12)
plt.gca().set_ylabel('Sub-category', fontsize=12)
plt.gca().set_title('Distribution of sub-category', fontsize=14)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.show()

In [ ]:
# Create a list to store sub-category
sub_category_lst = df['sub_category'].unique()
# Create a list to store sub-category ID
sub_category_id_lst = list(range(1, len(sub_category_lst)+1))
# Add sub-category ID collumn to df
df['sub_category_id'] = df['sub_category'].apply(lambda x: sub_category_id_lst[sub_category_lst.tolist().index(x)])
# Combine 'sub_category_id' and 'product_id' into 'item_id'

In [ ]:
df['sub_category_id'].head()

In [ ]:
df['sub_category_id'].tail()

### Text Cleaning

In [ ]:
def text_preprocessing(df, _input='product_name_description', _output='description_processed'):
  INPUT_ = ['product_name_description',]
  # Add suffix '_processed' to the output column name from INPUT_
  OUTPUT_ = [x + '_processed' for x in INPUT_]
  # Apply text preprocessing to the input column
  for _input, _output in zip(INPUT_, OUTPUT_):
    df[_output] = df[_input].apply(lambda x: preprocess_lib.process_text(x))
    df[_output] = df[_output].apply(lambda x: preprocess_lib.covert_unicode(x))
    df[_output] = df[_output].apply(lambda x: preprocess_lib.process_postag_thesea(x))
    df[_output] = df[_output].apply(lambda x: preprocess_lib.process_special_word(x, SPECIAL_WORDS))
    df[_output] = df[_output].apply(lambda x: preprocess_lib.remove_stopword(x))
  return df

In [ ]:
# Create a cluster
cluster = LocalCluster()
client = Client(cluster)
dask.config.set(scheduler='multiprocessing')

# Create a dask dataframe from the pandas dataframe
# e.g: CPUC 6 cores => select 4 cores * (2 ~ 3)
_CORES_NUM = os.cpu_count() - 2
_CORES_COEFF = 2
_PARTITION_NUM = _CORES_NUM * _CORES_COEFF
print('_CORES_NUM: ', _CORES_NUM)
print('_PARTITION_NUM: ', _PARTITION_NUM)

ddf = dd.from_pandas(df, npartitions=_PARTITION_NUM)
ddf = ddf.map_partitions(text_preprocessing)

# Add time to check how long it takes to process
start_time = time.time()
# Convert the dask dataframe to pandas dataframe
print('Converting dask dataframe to pandas dataframe...')
with ProgressBar():
  df = ddf.compute(num_workers=_CORES_NUM)
# Free up memory
del ddf
print('Time to process: ', (time.time() - start_time) / 60, ' mins')

In [ ]:
df.shape

In [ ]:
# Reset index
df.reset_index(drop=True, inplace=True)

In [ ]:
df[['sub_category_id', 'product_id', 'product_name', 'description', 'product_name_description_processed']].head(20)

In [ ]:
df[['sub_category_id', 'product_id', 'product_name', 'description', 'product_name_description_processed']].tail(20)

## Save processed data to csv file

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
# Save processed data to csv file
df.to_csv(ProcessedFilePath, index=False, encoding='utf8')